###  v1.1. optimized version but still in single class

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# для TimeSeriesPreprocessorManager
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
from prophet import Prophet
from statsmodels.tsa.seasonal import STL, seasonal_decompose
from statsmodels.tsa.stattools import adfuller, acf, pacf, kpss
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.ar_model import AutoReg
from scipy.stats import zscore
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
import pmdarima as pm  # INSTALL TO REQ
from scipy import signal
from scipy.stats import zscore, probplot, pearsonr
from joblib import Parallel, delayed
from tbats import TBATS
import holidays
from sklearn.preprocessing import QuantileTransformer
from tbats import TBATS
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler




from prophet.diagnostics import cross_validation, performance_metrics
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

Importing plotly failed. Interactive plots will not work.


# Получение входных данных

In [3]:
CLICKHOUSE_HOST='localhost'
CLICKHOUSE_PORT=8123
CLICKHOUSE_USER='CLICKHOUSE_USER' # INPUT YOUR CREDENTIAL 
CLICKHOUSE_USER='CLICKHOUSE_USER' 
CLICKHOUSE_PASSWORD='CLICKHOUSE_PASSWORD'

In [4]:
# receiving data

import clickhouse_connect
import os
import sys


client = clickhouse_connect.get_client(
                host=CLICKHOUSE_HOST,
                port=CLICKHOUSE_PORT,
                user=CLICKHOUSE_USER,
                password=CLICKHOUSE_PASSWORD
            )

In [5]:
# load time series from db
tablename = "CRYPTO.btcusdt_12h_raw"
ch_response = client.query(f"SELECT * FROM {tablename} ORDER BY Open_time")
ts_data = pd.DataFrame(ch_response.result_rows, columns=ch_response.column_names)

In [6]:
ts_data.head()

,Open_time,Open,High,Low,Close,Volume,Close_time,Quote_asset_volume,Number_of_trades,Taker_buy_base_asset_volume,Taker_buy_quote_asset_volume,ts_id,Source
0,2017-08-17 00:00:00,4261.48,4485.39,4261.32,4427.30,145.708747,2017-08-17 11:59:59,6.356955e+05,582,122.801360,5.367015e+05,btcusdt_12h,binance_api
1,2017-08-17 12:00:00,4436.06,4485.39,4200.74,4285.08,649.441630,2017-08-17 23:59:59,2.819075e+06,2845,493.447181,2.141515e+06,btcusdt_12h,binance_api
2,2017-08-18 00:00:00,4285.08,4371.52,4134.61,4340.31,720.722201,2017-08-18 11:59:59,3.085356e+06,3051,585.408001,2.508148e+06,btcusdt_12h,binance_api
3,2017-08-18 12:00:00,4320.52,4340.31,3938.77,4108.37,479.166063,2017-08-18 23:59:59,2.001602e+06,2182,387.460709,1.620975e+06,btcusdt_12h,binance_api
4,2017-08-19 00:00:00,4108.37,4184.69,3850.00,3957.60,298.518569,2017-08-19 11:59:59,1.214383e+06,1559,213.297785,8.706680e+05,btcusdt_12h,binance_api


In [7]:
ts_data.tail()

,Open_time,Open,High,Low,Close,Volume,Close_time,Quote_asset_volume,Number_of_trades,Taker_buy_base_asset_volume,Taker_buy_quote_asset_volume,ts_id,Source
6149,2026-01-16 12:00:00,95429.09,95809.08,94293.46,95550.94,6051.25953,2026-01-16 23:59:59,5.753072e+08,1962043,2867.23124,2.726688e+08,btcusdt_12h,binance_api
6150,2026-01-17 00:00:00,95550.94,95578.20,95021.67,95323.86,1694.35238,2026-01-17 11:59:59,1.614328e+08,459123,839.78958,8.001233e+07,btcusdt_12h,binance_api
6151,2026-01-17 12:00:00,95323.85,95639.45,95050.00,95147.77,2586.44899,2026-01-17 23:59:59,2.467292e+08,561404,1220.06325,1.163867e+08,btcusdt_12h,binance_api
6152,2026-01-18 00:00:00,95147.77,95295.39,94876.33,95168.06,2386.33318,2026-01-18 11:59:59,2.269771e+08,513805,1112.37410,1.058134e+08,btcusdt_12h,binance_api
6153,2026-01-18 12:00:00,95168.06,95376.24,94910.00,95183.55,1256.21349,2026-01-18 23:59:59,1.194810e+08,279502,566.66885,5.389214e+07,btcusdt_12h,binance_api


# Optimized version, but a single class without splitting into separate logical modules 

In [8]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import acf
from scipy.signal import find_peaks  # not used in optimized version
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator
from numba import njit

# -----------------------------------------------------------------------------
# Numba function for period selection based on autocorrelation values
# -----------------------------------------------------------------------------

@njit
def select_period(acf_values, min_period, max_period_loc, period_threshold): 
    n = acf_values.shape[0]
    best_period = 0
    best_acf = 0.0
    found_strong = False
    # Iterate through indices from min_period to max_period_loc (inclusive)
    for i in range(min_period, min(max_period_loc+1, n)):
        # Simple peak check: if value greater than previous and not less than next
        if i > 0 and i < n - 1:
            if acf_values[i] > acf_values[i - 1] and acf_values[i] >= acf_values[i + 1]:
                if acf_values[i] >= period_threshold and not found_strong:
                    best_period = i
                    found_strong = True
                elif not found_strong:
                    if acf_values[i] > best_acf:
                        best_acf = acf_values[i]
                        best_period = i
    return best_period



# -----------------------------------------------------------------------------
# PeriodicityEstimator class with optimized period selection logic
# -----------------------------------------------------------------------------

class PeriodicityEstimator(BaseEstimator): 
    def __init__(self, max_nlags_ratio=0.5, peak_prominence_threshold=0.1,
                 period_threshold=0.8, min_period=2, max_period=None):
        self.max_nlags_ratio = max_nlags_ratio
        self.peak_prominence_threshold = peak_prominence_threshold  # not used in optimized version
        self.period_threshold = period_threshold
        self.min_period = min_period
        self.max_period = max_period
        self.period_ = None

    def fit(self, X, y=None): 
        return self

    def predict(self, X): 
        period = self._find_period(X)
        self.period_ = period
        return period

    def _find_period(self, data): 
        n = len(data)
        max_nlags = int(self.max_nlags_ratio * n)
        max_period_loc = n // 2 if self.max_period is None else self.max_period
        acf_values = acf(data, nlags=max_nlags, fft=True)
        # Period selection using Numba function
        period = select_period(acf_values, self.min_period, max_period_loc, self.period_threshold)
        return period

# -----------------------------------------------------------------------------
# PeriodicityDetector class with additional downsampling in estimation
# -----------------------------------------------------------------------------

class PeriodicityDetector(BaseEstimator): 
    def __init__(self, min_period=2, max_period=None, cv=5, downsample_factor=10):
        self.min_period = min_period
        self.max_period = max_period
        self.cv = cv
        self.downsample_factor = downsample_factor
        self.best_estimator_ = None

    def fit(self, X, y=None): 
        param_grid = {
            'max_nlags_ratio': [0.1, 0.25, 0.5],
            'peak_prominence_threshold': [0.05, 0.1, 0.2],
            'period_threshold': [0.5, 0.6, 0.7, 0.8],
            'min_period': [self.min_period],
            'max_period': [self.max_period]
        }
        estimator = PeriodicityEstimator()
        grid_search = GridSearchCV(estimator, param_grid, scoring=self._score_period, cv=self.cv, n_jobs=-1)
        # Pass data as one-dimensional array
        grid_search.fit(X.to_numpy().ravel().reshape(-1, 1), y=[0] * len(X))
        self.best_estimator_ = grid_search.best_estimator_
        return self

    def _score_period(self, y, period_, X=None, **kwargs): 
        period = period_[0]
        if period == 0:
            return -np.inf
        try:
            # Downsampling to speed up STL
            X_ds = X[::self.downsample_factor]
            from statsmodels.tsa.seasonal import STL
            stl = STL(X_ds, period=period)
            res = stl.fit()
            return -res.resid.var()
        except Exception:
            return -np.inf

    def predict(self, X): 
        return self.best_estimator_.predict(X)

# -----------------------------------------------------------------------------
# Usage example (uncomment for testing)
# -----------------------------------------------------------------------------
# index = pd.date_range('1/1/2000', periods=365 * 10, freq='D')
# data = pd.Series(np.sin(2 * np.pi * index.dayofyear / 12) + np.random.randn(len(index)) / 5, index=index)
# detector = PeriodicityDetector(cv=20, downsample_factor=10)
# detector.fit(data)
# period = detector.predict(data)
# print(f"Optimal period: {period}")
# print(f"Best parameters: {detector.best_estimator_.get_params()}")

In [9]:
import numpy as np
import pandas as pd
import time
from scipy.stats import zscore, probplot
from statsmodels.tsa.stattools import adfuller, kpss, acf
from statsmodels.tsa.seasonal import STL
import statsmodels.api as sm
import holidays
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from joblib import Parallel, delayed
from numpy.lib.stride_tricks import as_strided
from numba import njit

# -----------------------------------------------------------------------------
# Numba-accelerated functions for rolling trend and diagonal averaging (Hankel averaging)
# -----------------------------------------------------------------------------

@njit
def rolling_slope(arr, window): 
    n = arr.shape[0]
    result = np.empty(n)
    result[:] = np.nan
    mean_i = (window - 1) / 2.0
    denom = 0.0
    for j in range(window):
        denom += (j - mean_i) ** 2
    for i in range(window - 1, n):
        s = 0.0
        for j in range(window):
            s += (j - mean_i) * arr[i - window + 1 + j]
        result[i] = s / denom
    return result

@njit
def hankel_diagonal_averaging(X): 
    window_length, K = X.shape
    N = window_length + K - 1
    reconstructed = np.zeros(N)
    counts = np.zeros(N)
    for i in range(window_length):
        for j in range(K):
            reconstructed[i + j] += X[i, j]
            counts[i + j] += 1.0
    for i in range(N):
        reconstructed[i] /= counts[i]
    return reconstructed

# -----------------------------------------------------------------------------
# TimeSeriesPreprocessorManager class with optimizations and logging
# -----------------------------------------------------------------------------

class TimeSeriesPreprocessorManager:
    """
    Class for time series preprocessing.

    Main purpose:
      - Prepare data for machine learning models and neural networks.
      - Perform cleaning, missing value imputation, outlier removal, scaling,
        decomposition, lag feature generation, and additional feature engineering.

    Usage example:
        preprocessor = TimeSeriesPreprocessorManager(
            interval='1d',
            timestamp_column='Open_time',
            target_column='Open',
            scaler='robust',
            lag_features=5
        )
        X, y = preprocessor.process(raw_df)
    """
    
    def __init__(self, interval, timestamp_column='Open_time', target_column='Open', additional_columns=None, scaler='robust', lag_features=5):
        self.interval = interval
        self.timestamp_column = timestamp_column
        self.target_column = target_column
        self.additional_columns = additional_columns if additional_columns else []
        self.scaler_type = scaler
        self.lag_features = lag_features
        self.original_data = None

    def analyze_series(self, data): 
        analysis = {}
        analysis['missing_values'] = data.isNone().sum()
        z_scores = zscore(data.dropna())
        analysis['zscore_outliers'] = int((np.abs(z_scores) > 3).sum())
        q1 = data.quantile(0.25)
        q3 = data.quantile(0.75)
        iqr = q3 - q1
        analysis['iqr_outliers'] = int(((data < (q1 - 1.5 * iqr)) | (data > (q3 + 1.5 * iqr))).sum())
        median_val = data.median()
        mad = np.median(np.abs(data - median_val))
        modified_z = 0.6745 * (data - median_val) / mad if mad != 0 else np.zeros_like(data)
        analysis['mad_outliers'] = int((np.abs(modified_z) > 3.5).sum())
        adf_result = adfuller(data.dropna())
        analysis['adf_pvalue'] = adf_result[1]
        stationary_adf = adf_result[1] < 0.05
        try:
            kpss_result = kpss(data.dropna(), regression='c', nlags="auto")
            analysis['kpss_pvalue'] = kpss_result[1]
            stationary_kpss = kpss_result[1] > 0.05
        except Exception as e:
            analysis['kpss_pvalue'] = np.nan
            stationary_kpss = False
        window = max(30, len(data) // 10)
        rolling_mean = data.rolling(window=window, min_periods=1).mean()
        rolling_std = data.rolling(window=window, min_periods=1).std()
        mean_cv = rolling_mean.std() / rolling_mean.mean() if rolling_mean.mean() != 0 else np.inf
        std_cv = rolling_std.std() / rolling_std.mean() if rolling_std.mean() != 0 else np.inf
        analysis['rolling_mean_cv'] = mean_cv
        analysis['rolling_std_cv'] = std_cv
        autocorr_lag1 = data.autocorr(lag=1)
        analysis['lag1_autocorrelation'] = autocorr_lag1
        qq = probplot(data.dropna(), dist="norm", plot=None)
        analysis['qq_correlation'] = qq[1][2]
        rolling_threshold = 0.2
        self._is_stationary = (stationary_adf and stationary_kpss and (mean_cv < rolling_threshold) and (std_cv < rolling_threshold))
        analysis['stationary'] = self._is_stationary
        return analysis

    def detrend_series(self, data, method='linear'): 
        if method == 'linear':
            x = np.arange(len(data))
            coeffs = np.polyfit(x, data.values, 1)
            trend = np.polyval(coeffs, x)
            detrended = data - trend
            return detrended, pd.Series(trend, index=data.index)
        elif method == 'difference':
            detrended = data.diff().dropna()
            return detrended, data.iloc[0]
        else:
            raise ValueError("Unsupported detrending method. Choose 'linear' or 'difference'.")

    def inverse_detrend(self, detrended_data, trend_component, method='linear'): 
        if method == 'linear':
            return detrended_data + trend_component
        elif method == 'difference':
            return detrended_data.cumsum() + trend_component
        else:
            raise ValueError("Unsupported inverse detrending method. Choose 'linear' or 'difference'.")

    def fill_missing_values(self, data, method='linear'):
        return data.interpolate(method=method).bfill().ffill()

    def _optimize_contamination(self, data, method='isolation_forest'): 
        if not isinstance(data, pd.Series):
            data = data[self.target_column]
        iqr = data.quantile(0.75) - data.quantile(0.25)
        lower_bound = data.quantile(0.25) - 1.5 * iqr
        upper_bound = data.quantile(0.75) + 1.5 * iqr
        estimated_outliers = ((data < lower_bound) | (data > upper_bound)).sum()
        estimated_contamination = max(estimated_outliers / len(data), 0.001)
        contamination_range = np.geomspace(max(0.0001, estimated_contamination / 2), 
                                            min(0.51, estimated_contamination * 2), 10)
        best_contamination = 0.001
        best_score = float('inf')
    
        def evaluate_contamination(contamination): 
            try:
                if method == 'isolation_forest':
                    processed_data = self.remove_outliers_with_isolation_forest(data.copy(), contamination)
                elif method == 'lof':
                    processed_data = self.remove_outliers_with_lof(data.copy(), contamination)
                else:
                    raise ValueError("Unsupported method for contamination optimization.")
                residual_diff = data - processed_data
                score = np.mean(np.abs(residual_diff))
                autocorr_score = residual_diff.autocorr()
                final_score = score * (1 - abs(autocorr_score))
                return contamination, final_score
            except Exception as e:
                print(f"Error evaluating contamination {contamination}: {e}")
                return contamination, float('inf')
    
        results = Parallel(n_jobs=-1)(delayed(evaluate_contamination)(cont) for cont in contamination_range)
        for contamination, score in results:
            if score < best_score:
                best_score = score
                best_contamination = contamination
        return best_contamination

    def remove_outliers_with_isolation_forest(self, data, contamination=None):
        if contamination is None:
            contamination = self._optimize_contamination(data, method='isolation_forest')
        reshaped_data = data.values.reshape(-1, 1)
        isol_forest = IsolationForest(contamination=contamination)
        outliers = isol_forest.fit_predict(reshaped_data)
        median_value = np.median(data[~np.isin(outliers, -1)])
        data[outliers == -1] = median_value
        return data

    def remove_outliers_with_lof(self, data, contamination=None, n_neighbors=None): 
        if contamination is None:
            contamination = self._optimize_contamination(data, method='lof')
        if n_neighbors is None:
            n_neighbors = min(20, max(5, int(len(data) * 0.05)))
        reshaped_data = data.values.reshape(-1, 1)
        lof = LocalOutlierFactor(contamination=contamination, n_neighbors=n_neighbors)
        outliers = lof.fit_predict(reshaped_data)
        median_value = np.median(data[outliers == 1])
        data[outliers == -1] = median_value
        return data

    def decompose_and_remove_outliers(self, data, method='isolation_forest', contamination=None): 
        trend, seasonal, residual = self.decompose_series(data, method='STL')
        if method == 'isolation_forest':
            residual_cleaned = self.remove_outliers_with_isolation_forest(residual, contamination)
        elif method == 'lof':
            residual_cleaned = self.remove_outliers_with_lof(residual, contamination)
        else:
            raise ValueError("Unsupported method for outlier removal.")
        cleaned_series = trend + seasonal + residual_cleaned
        return cleaned_series

    def remove_outliers(self, data, method='combined', contamination=None): 
        if method == 'combined':
            remove_outliers_method = 'isolation_forest' if len(data) > 1000 else 'lof'
            return self.decompose_and_remove_outliers(data.copy(), method=remove_outliers_method, contamination=contamination)
        elif method == 'isolation_forest':
            return self.remove_outliers_with_isolation_forest(data.copy(), contamination)
        elif method == 'lof':
            return self.remove_outliers_with_lof(data.copy(), contamination)
        else:
            raise ValueError("Unsupported outlier removal method.")

    def determine_period(self, data): 
        # Use PeriodicityDetector, ACF and FFT
        # (Assumes PeriodicityDetector is defined somewhere in the environment)
        detector = PeriodicityDetector(min_period=2, max_period=None, cv=10)
        detector.fit(data)
        period_detector = detector.predict(data)
        period_candidates = []
        if period_detector > 1:
            print(f"[LOG] Applying PeriodicityDetector: {period_detector}")
            period_candidates.append(period_detector)
        acf_vals = acf(data.dropna(), nlags=min(100, len(data)//2))
        for lag, val in enumerate(acf_vals[1:], start=1):
            if val > 0.3:
                print(f"[LOG] Applying ACF: selected lag={lag}")
                period_candidates.append(lag)
                break
        fft_data = data.dropna().values
        fft_vals = np.abs(np.fft.rfft(fft_data))
        freqs = np.fft.rfftfreq(len(fft_data), d=1)
        if len(fft_vals) > 1:
            index_candidate = np.argmax(fft_vals[1:]) + 1
            if index_candidate >= len(freqs):
                index_candidate = len(freqs) - 1
            dominant_freq = freqs[index_candidate]
            default_max_period = {
                '1h': 168,    # 24*7 - week for hourly data
                '12h': 60,    # approximately 30 days
                '1d': 182,    # approximately half year for daily data
                '3d': 60,     # approximately 180 days/3 ≈ 60 observations for 3-day interval
                '1w': 52,     # 52 weeks (year) for weekly data
                '1M': 12      # 12 months for monthly data
            }
            max_allowed_period = default_max_period.get(self.interval, len(data)/2)
            if dominant_freq > 0:
                period_fft = int(round(1 / dominant_freq))
                if period_fft < max_allowed_period:
                    #print(f"[LOG] Applying FFT: period={period_fft}")
                    period_candidates.append(period_fft)
                else:
                    print(f"[LOG] Ignoring FFT candidate: {period_fft} (exceeds threshold)")
        if period_candidates:
            period = int(np.round(np.median(period_candidates)))
        else:
            period = 0
        if period < 2:
            print("[LOG] Applying Fallback for period")
            default_periods = {'1h': 24, '12h': 14, '1d': 7, '3d': 7, '1w': 52, '1M': 12}
            period = default_periods.get(self.interval, 1)
        print(f"[LOG] Determined period: {period}, candidates: {period_candidates}")
        return period, period_candidates

    def get_seasonality_config(self, interval, data_length): 
        configs = {
            '1h': {'daily': True, 'weekly': False, 'yearly': False},
            '12h': {'daily': True, 'weekly': True, 'yearly': False},
            '1d': {'daily': False, 'weekly': True, 'yearly': True},
            '3d': {'daily': False, 'weekly': True, 'yearly': True},
            '1w': {'daily': False, 'weekly': False, 'yearly': True},
            '1M': {'daily': False, 'weekly': False, 'yearly': True}
        }
        config = configs.get(interval, {'daily': False, 'weekly': False, 'yearly': False})
        if data_length < 365:
            config['yearly'] = False
        return config

    def decompose_series(self, data, method='auto', freq=None, model='auto'): 
        if not isinstance(data.index, pd.DatetimeIndex):
            raise ValueError("Data must have a DatetimeIndex for seasonal decomposition.")
        adf_pvalue = adfuller(data.dropna())[1]
        model_type = 'additive' if adf_pvalue < 0.05 else 'multiplicative'
        period, period_candidates = self.determine_period(data)
        method = self._determine_method_auto(data, period, period_candidates) if method == 'auto' else method
        decomposition_methods = {
            'STL': self._decompose_stl,
            'TBATS': self._decompose_tbats,
            'SSA': self._decompose_ssa,
            'fourier': self._decompose_fourier,
            'prophet': self._decompose_prophet
        }
        if method in decomposition_methods:
            return decomposition_methods[method](data, period, period_candidates, model_type)
        else:
            raise ValueError(f"Unsupported decomposition method: {method}")

    def _determine_method_auto(self, data, period, candidates): 
        if period < 2:
            return 'SSA'
        methods = ['STL', 'SSA', 'fourier']
        quality = {}
        for method in methods:
            result = getattr(self, f'_decompose_{method.lower()}')(data, period, candidates)
            quality[method] = np.nanmean(np.abs(result[2].dropna()))
        return min(quality, key=quality.get)

    def _decompose_stl(self, data, period, candidates, model_type=None): 
        print('[LOG] Decomposition: STL method selected')
        stl = STL(data, period=period, robust=True)
        result = stl.fit()
        return result.trend, result.seasonal, result.resid

    def _decompose_tbats(self, data, period, candidates, model_type=None): 
        print('[LOG] Decomposition: TBATS method selected')
        seasonal_periods = candidates if len(candidates) > 1 else [period]
        if self.interval == '1h':
            seasonal_periods = [24, 168]
        print(f"[LOG] TBATS: seasonal periods {seasonal_periods}")
        estimator = TBATS(seasonal_periods=seasonal_periods)
        tbats_model = estimator.fit(data)
        fitted = tbats_model.y_hat
        try:
            seasonal_array = np.sum(tbats_model.seasonal_components_, axis=1)
        except AttributeError:
            seasonal_array = np.zeros_like(fitted)
        trend_array = fitted - seasonal_array
        resid_array = data.values.flatten() - fitted
        trend = pd.Series(trend_array, index=data.index)
        seasonal = pd.Series(seasonal_array, index=data.index)
        resid = pd.Series(resid_array, index=data.index)
        return trend, seasonal, resid

    def _decompose_ssa(self, data, period, candidates, model_type=None): 
        print('[LOG] Decomposition: SSA method selected')
        return self.ssa_decompose(data, window_length=period, variance_threshold=0.9)

    def _decompose_fourier(self, data, period, candidates, model_type=None): 
        print('[LOG] Decomposition: FOURIER method selected')
        # Add timing log to verify harmonic loop
        t_start = time.perf_counter()
        result = self.fourier_decompose(data, period=period)
        t_end = time.perf_counter()
        print(f"[LOG] FOURIER-decompose time: {(t_end - t_start)*1000:.2f} ms")
        return result

    def _decompose_prophet(self, data, period, candidates, model_type=None): 
        print('[LOG] Decomposition: Prophet method selected')
        df = pd.DataFrame(data)
        df.reset_index(inplace=True)
        df.columns = ['ds', 'y']
        data_length = len(df)
        seasonality_config = self.get_seasonality_config(self.interval, data_length)
        components = [comp for comp, flag in seasonality_config.items() if flag]
        model_prophet = Prophet(
            daily_seasonality=seasonality_config.get('daily', False),
            weekly_seasonality=seasonality_config.get('weekly', False),
            yearly_seasonality=seasonality_config.get('yearly', False),
            changepoint_prior_scale=0.05,
            seasonality_prior_scale=10,
            seasonality_mode='multiplicative'
        )
        model_prophet.fit(df)
        forecast = model_prophet.predict(df)
        trend_series = pd.Series(forecast['trend'].values, index=data.index)
        seasonal_series = pd.Series(forecast[components].sum(axis=1).values, index=data.index)
        resid_series = pd.Series((df['y'] - forecast['trend'] - forecast[components].sum(axis=1)).values, index=data.index)
        return trend_series, seasonal_series, resid_series

    def fourier_decompose(self, data, period): 
        # Pre-compute sine and cosine features for max_k
        data_series = pd.Series(data.values.flatten(), index=data.index)
        df = pd.DataFrame({'y': data_series.values}, index=data_series.index)
        t = np.arange(len(df))
        max_k = 5
        const = np.ones_like(t)
        harmonics = np.arange(1, max_k+1)
        sin_full = np.sin(2 * np.pi * np.outer(t, harmonics) / period)
        cos_full = np.cos(2 * np.pi * np.outer(t, harmonics) / period)
        best_aic = np.inf
        best_model = None
        best_X = None
        for k in range(1, max_k+1):
            X = np.column_stack((const, t, sin_full[:, :k], cos_full[:, :k]))
            X = sm.add_constant(X, has_constant='add')
            model_fit = sm.OLS(df['y'], X).fit()
            if model_fit.aic < best_aic:
                best_aic = model_fit.aic
                best_model = model_fit
                best_X = X.copy()
        params = best_model.params
        trend_values = best_X[:, 1] * params[1] + best_X[:, 0]
        seasonal_values = best_X[:, 2:] @ params[2:]
        return (pd.Series(trend_values, index=df.index),
                pd.Series(seasonal_values, index=df.index),
                data_series - pd.Series(trend_values, index=df.index) - pd.Series(seasonal_values, index=df.index))

    def ssa_decompose(self, data, window_length=None, variance_threshold=0.9): 
        series = data.values.flatten() if isinstance(data, pd.Series) else np.array(data).flatten()
        N = series.shape[0]
        if window_length is None:
            period, _ = self.determine_period(data)
            window_length = period if period > 1 else N // 2
        K = N - window_length + 1
        X = as_strided(series, shape=(window_length, K), strides=(series.strides[0], series.strides[0])).copy()
        U, s, Vt = np.linalg.svd(X, full_matrices=False)
        total_variance = np.sum(s**2)
        explained_variance = np.cumsum(s**2) / total_variance
        r = np.searchsorted(explained_variance, variance_threshold) + 1
        X_reconstructed = np.zeros_like(X)
        for i in range(r):
            X_reconstructed += s[i] * np.outer(U[:, i], Vt[i, :])
        reconstructed = hankel_diagonal_averaging(X_reconstructed)
        if r >= 2:
            X_trend = s[0] * np.outer(U[:, 0], Vt[0, :]) + s[1] * np.outer(U[:, 1], Vt[1, :])
            trend_reconstructed = hankel_diagonal_averaging(X_trend)
            seasonal_reconstructed = reconstructed - trend_reconstructed
        else:
            trend_reconstructed = reconstructed
            seasonal_reconstructed = np.zeros_like(reconstructed)
        residual = series - reconstructed
        if hasattr(data, 'index'):
            trend_reconstructed = pd.Series(trend_reconstructed, index=data.index)
            seasonal_reconstructed = pd.Series(seasonal_reconstructed, index=data.index)
            residual = pd.Series(residual, index=data.index)
        return trend_reconstructed, seasonal_reconstructed, residual

    def scale_data(self, data): 
        initial_stats = {
            'min': data.min(),
            'max': data.max(),
            'mean': data.mean(),
            'std': data.std(),
            'median': data.median(),
            'IQR': data.quantile(0.75) - data.quantile(0.25)
        }
        #print("[LOG] Initial data stats:", initial_stats)
        reshaped_data = data.values.reshape(-1, 1)
        if self.scaler_type == 'minmax':
            scaler = MinMaxScaler()
        elif self.scaler_type == 'standard':
            scaler = StandardScaler()
        elif self.scaler_type == 'robust':
            scaler = RobustScaler()
        elif self.scaler_type == 'quantile':
            scaler = QuantileTransformer(output_distribution='normal')
        elif self.scaler_type == 'log':
            scaled_data = np.log1p(reshaped_data)
            self.custom_inverse = lambda x: np.expm1(x)
            scaled_data = scaled_data.flatten()
            log_stats = {
                'min': scaled_data.min(),
                'max': scaled_data.max(),
                'mean': scaled_data.mean(),
                'std': scaled_data.std()
            }
            print("[LOG] Data after log scaling stats:", log_stats)
            return scaled_data
        else:
            raise ValueError("Unsupported scaler type. Choose from: 'minmax', 'standard', 'robust', 'quantile', 'log'.")
    
        scaled_data = scaler.fit_transform(reshaped_data).flatten()
        after_stats = {
            'min': scaled_data.min(),
            'max': scaled_data.max(),
            'mean': scaled_data.mean(),
            'std': scaled_data.std()
        }
        print("[LOG] Scaled data stats:", after_stats)
        self.scaler = scaler
        return scaled_data

    def inverse_scale(self, scaled_data): 
        if self.scaler_type == 'log' and hasattr(self, 'custom_inverse'):
            reshaped_data = scaled_data.reshape(-1, 1)
            inversed = self.custom_inverse(reshaped_data)
            return inversed.flatten()
        else:
            reshaped_data = scaled_data.reshape(-1, 1)
            return self.scaler.inverse_transform(reshaped_data).flatten()

    def add_features(self, df, windows=None, holiday_country='US'): 
        t0 = time.perf_counter()
        df = df.copy()
        if not np.issubdtype(df.index.dtype, np.datetime64):
            df.index = pd.to_datetime(df.index, errors='coerce')
        base_col = f'{self.target_column}_scaled' if f'{self.target_column}_scaled' in df.columns else self.target_column
        features = df.copy()
        features[base_col] = df[base_col]
        # Vectorized lag feature creation
        lag_config = {
            '1h': [1, 12, 24, 168],
            '12h': [1, 2, 3, 14],
            '1d': [1, 7, 14, 30],
            '3d': [1, 2, 7],
            '1w': [1, 2, 4],
            '1M': [1, 3, 6, 12]
        }
        lags = lag_config.get(self.interval, list(range(1, self.lag_features + 1)))
        lag_features = pd.concat([df[base_col].shift(lag).rename(f'lag_{lag}') for lag in lags], axis=1)
        features = pd.concat([features, lag_features], axis=1)
        default_windows = {
            '1h': [3, 6, 12, 24],
            '12h': [2, 4, 6, 14],
            '1d': [3, 7, 14, 30],
            '3d': [2, 4, 7],
            '1w': [2, 4, 6],
            '1M': [3, 6, 12]
        }
        if windows is None:
            windows = default_windows.get(self.interval, [3, 5, 7])
        for window in windows:
            if window > len(df):
                continue
            features[f'roll_mean_{window}'] = df[base_col].rolling(window=window, min_periods=1).mean()
            features[f'roll_std_{window}'] = df[base_col].rolling(window=window, min_periods=1).std()
            features[f'roll_min_{window}'] = df[base_col].rolling(window=window, min_periods=1).min()
            features[f'roll_max_{window}'] = df[base_col].rolling(window=window, min_periods=1).max()
            features[f'roll_median_{window}'] = df[base_col].rolling(window=window, min_periods=1).median()
            if window > 2:
                features[f'roll_skew_{window}'] = df[base_col].rolling(window=window, min_periods=1).skew()
            if window > 3:
                features[f'roll_kurtosis_{window}'] = df[base_col].rolling(window=window, min_periods=1).kurt() 
        features['first_derivative'] = df[self.target_column].diff()
        features['second_derivative'] = df[self.target_column].diff(2)
        for window in windows:
            features[f'rolling_trend_{window}'] = rolling_slope(df[self.target_column].values, window)
        features['returns'] = df[self.target_column].pct_change()
        vol_window = windows[len(windows)//2] if windows else 7
        features['volatility'] = features['returns'].rolling(window=vol_window, min_periods=1).std()
        
        try:
            years = pd.to_datetime(df.index).year.unique().tolist()
            country_holidays = holidays.CountryHoliday(holiday_country, years=years, expand=True, observed=True)
            features['is_holiday'] = pd.Series(df.index.map(lambda d: d.date() in country_holidays), index=df.index)
        except Exception:
            features['is_holiday'] = False    
        features.to_csv('features.csv', index=False)
        features = features.dropna()
        t1 = time.perf_counter()
        return features

    def process(self, df): 
        try:
            t_start = time.perf_counter()
            df = df.copy()
            if self.timestamp_column not in df.columns:
                raise ValueError(f"Timestamp column '{self.timestamp_column}' missing.")
            if not np.issubdtype(df[self.timestamp_column].dtype, np.datetime64):
                df[self.timestamp_column] = pd.to_datetime(df[self.timestamp_column], errors='coerce')
            self.original_data = df.copy()
            df.set_index(self.timestamp_column, inplace=True)
            for col in [self.target_column] + self.additional_columns:
                df[col] = self.fill_missing_values(df[col])
            _ = self.analyze_series(df[self.target_column])  # Analysis for diagnostics (no output)
            df[f'{self.target_column}_removed_outliers'] = self.remove_outliers(df[self.target_column])
            scaled_series = self.scale_data(df[f'{self.target_column}_removed_outliers'])
            df[f'{self.target_column}_scaled'] = scaled_series
            df = self.add_features(df)
            df.sort_index(inplace=True)
            t_end = time.perf_counter()
            print(f"Pipeline finished. Observations: {len(df)}, Total time: {(t_end-t_start):.2f} s")
            y = df[f'{self.target_column}_scaled']
            X = df.drop(columns=[f'{self.target_column}_scaled'])            
            return X, y
        except Exception as e:
            raise Exception(f'[ml_manager/TimeSeriesPreprocessorManager] error: {e}')

In [11]:
# Create instance of TimeSeriesPreprocessorManager class
ts_preprocessor_manager = TimeSeriesPreprocessorManager('12h')
X, y = ts_preprocessor_manager.process(ts_data)

/var/folders/gb/tqzzyhfj48zf1dmpwjhzh_g40000gn/T/ipykernel_67597/1013231189.py:99: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_result = kpss(data.dropna(), regression='c', nlags="auto")


[LOG] Applying ACF: selected lag=1
[LOG] Ignoring FFT candidate: 3077 (exceeds threshold)
[LOG] Applying Fallback for period
[LOG] Determined period: 14, candidates: [1]
[LOG] Decomposition: STL method selected


Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


[LOG] Scaled data stats: {'min': -0.4956435849448443, 'max': 1.9788124482795066, 'mean': 0.2052625923849962, 'std': 0.6769338149376591}
Pipeline finished. Observations: 6140, Total time: 4.76 s


In [12]:
%timeit ts_preprocessor_manager.process(ts_data)

/var/folders/gb/tqzzyhfj48zf1dmpwjhzh_g40000gn/T/ipykernel_67597/1013231189.py:99: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_result = kpss(data.dropna(), regression='c', nlags="auto")


[LOG] Applying ACF: selected lag=1
[LOG] Ignoring FFT candidate: 3077 (exceeds threshold)
[LOG] Applying Fallback for period
[LOG] Determined period: 14, candidates: [1]
[LOG] Decomposition: STL method selected


Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


[LOG] Scaled data stats: {'min': -0.4974152791818321, 'max': 1.9865186856425734, 'mean': 0.20491798290368937, 'std': 0.6786758593977295}
Pipeline finished. Observations: 6140, Total time: 2.40 s


/var/folders/gb/tqzzyhfj48zf1dmpwjhzh_g40000gn/T/ipykernel_67597/1013231189.py:99: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_result = kpss(data.dropna(), regression='c', nlags="auto")


...

[LOG] Applying ACF: selected lag=1
[LOG] Ignoring FFT candidate: 3077 (exceeds threshold)
[LOG] Applying Fallback for period
[LOG] Determined period: 14, candidates: [1]
[LOG] Decomposition: STL method selected
[LOG] Scaled data stats: {'min': -0.49703005500691144, 'max': 1.9849802226044058, 'mean': 0.2052480813500624, 'std': 0.6783630927780284}
Pipeline finished. Observations: 6140, Total time: 1.70 s


/var/folders/gb/tqzzyhfj48zf1dmpwjhzh_g40000gn/T/ipykernel_67597/1013231189.py:99: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_result = kpss(data.dropna(), regression='c', nlags="auto")


[LOG] Applying ACF: selected lag=1
[LOG] Ignoring FFT candidate: 3077 (exceeds threshold)
[LOG] Applying Fallback for period
[LOG] Determined period: 14, candidates: [1]
[LOG] Decomposition: STL method selected
[LOG] Scaled data stats: {'min': -0.49564392414993436, 'max': 1.9788138025253301, 'mean': 0.2053218522011902, 'std': 0.677025304707114}
Pipeline finished. Observations: 6140, Total time: 1.85 s


/var/folders/gb/tqzzyhfj48zf1dmpwjhzh_g40000gn/T/ipykernel_67597/1013231189.py:99: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  kpss_result = kpss(data.dropna(), regression='c', nlags="auto")


[LOG] Applying ACF: selected lag=1
[LOG] Ignoring FFT candidate: 3077 (exceeds threshold)
[LOG] Applying Fallback for period
[LOG] Determined period: 14, candidates: [1]
[LOG] Decomposition: STL method selected
[LOG] Scaled data stats: {'min': -0.49689234817383865, 'max': 1.9844302652378172, 'mean': 0.20481401254059484, 'std': 0.6780442935186475}
Pipeline finished. Observations: 6140, Total time: 2.15 s
2.08 s ± 719 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Error evaluating contamination 0.51: The 'contamination' parameter of IsolationForest must be a str among {'auto'} or a float in the range (0.0, 0.5]. Got 0.51 instead.
Error evaluating contamination 0.51: The 'contamination' parameter of IsolationForest must be a str among {'auto'} or a float in the range (0.0, 0.5]. Got 0.51 instead.
Error evaluating contamination 0.51: The 'contamination' parameter of IsolationForest must be a str among {'auto'} or a float in the range (0.0, 0.5]. Got 0.51 instead.
Error evaluating co